In [2]:
api_key="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"

In [9]:
import requests
import json

API_KEY = "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"
URL     = "https://openrouter.ai/api/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type":  "application/json",
}

payload = {
    "model":    "openai/gpt-3.5-turbo",
    "messages": [{"role":"user", "content":"ping"}]
}

resp = requests.post(URL, headers=headers, json=payload)
print(resp.status_code, resp.text)


200 {"id":"gen-1750627871-7VwDMe6A8TSxRKFJVEVg","provider":"OpenAI","model":"openai/gpt-3.5-turbo","object":"chat.completion","created":1750627871,"choices":[{"logprobs":null,"finish_reason":"stop","native_finish_reason":"stop","index":0,"message":{"role":"assistant","content":"Pong! How can I assist you today?","refusal":null,"reasoning":null}}],"system_fingerprint":null,"usage":{"prompt_tokens":8,"completion_tokens":10,"total_tokens":18,"prompt_tokens_details":{"cached_tokens":0},"completion_tokens_details":{"reasoning_tokens":0}}}


In [10]:
import requests
import json
import time
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm

# ←––––– Your OpenRouter key here ––––→
API_KEY = "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"
URL     = "https://openrouter.ai/api/v1/chat/completions"
MODEL   = "openai/gpt-3.5-turbo"

# Mapping category names ↔ integer labels
label_map = {
    "Conflict-related": 1,
    "Protests and Public Reactions": 2,
    "Political Statements": 3,
    "Irrelevant": 4
}
reverse_label_map = {v: k for k, v in label_map.items()}

# Load your 200-article test set
with open("articles_test.json") as f:
    articles = json.load(f)

def create_prompt(article):
    txt = article["title"]
    if article.get("description"):
        txt += "\n\n" + article["description"]
    txt += "\n\n" + article["text"]
    return f"""You are a news classification assistant.

Classify the following news article into ONE of these categories:
1. Conflict-related
2. Political Statements
3. Protests and Public Reactions
4. Irrelevant

Return ONLY the category name (no extra text).

Article:
{txt}
"""

def classify_article(article):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model":    MODEL,
        "messages": [{"role":"user", "content": create_prompt(article)}],
        "temperature": 0.0,
    }
    r = requests.post(URL, headers=headers, json=payload)
    if r.status_code != 200:
        print("Error", r.status_code, r.text)
        return -1
    reply = r.json()["choices"][0]["message"]["content"].strip().lower()
    for cat, idx in label_map.items():
        if cat.lower() in reply:
            return idx
    return -1

# Run classification
y_true, y_pred = [], []
for art in tqdm(articles, desc="Classifying 200 articles"):
    y_true.append(art["label"])
    y_pred.append(classify_article(art))
    time.sleep(1.2)   # to respect rate limits

# Evaluate
print("\n--- Classification Report ---")
print(classification_report(
    y_true, y_pred,
    target_names=[reverse_label_map[i] for i in sorted(reverse_label_map)]
))
print("Macro F1 score:", f1_score(y_true, y_pred, average="macro"))


Classifying 200 articles: 100%|██████████| 200/200 [07:43<00:00,  2.32s/it]


--- Classification Report ---
                               precision    recall  f1-score   support

             Conflict-related       0.45      0.98      0.62        50
Protests and Public Reactions       0.88      0.45      0.60        33
         Political Statements       0.21      0.43      0.29        35
                   Irrelevant       0.75      0.04      0.07        82

                     accuracy                           0.41       200
                    macro avg       0.57      0.47      0.39       200
                 weighted avg       0.60      0.41      0.33       200

Macro F1 score: 0.39295848220815305


In [11]:
# after your run…
total = len(y_pred)
errors = y_pred.count(-1)
success = total - errors

print(f"Total articles processed: {total}")
print(f"✅ Successful classifications: {success}")
print(f"❌ Errors (returned -1):   {errors}")


Total articles processed: 200
✅ Successful classifications: 200
❌ Errors (returned -1):   0


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


IndexError: index out of range in self

## Baseline BERT

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import BertTokenizerFast, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
import json

# 1) Load your full labeled dataset (must contain 'title', 'text', 'label')
with open("articles_test.json") as f:
    data = json.load(f)
texts  = [d['title'] + " " + d['text'] for d in data]
labels = [d['label'] - 1 for d in data]  # convert 1–4 → 0–3

# 2) Split into train/validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# 3) Tokenizer & Dataset definition
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length)
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_texts, train_labels, tokenizer)
val_dataset   = NewsDataset(val_texts, val_labels, tokenizer)

# 4) Model & optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model  = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

# 5) DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16)

# 6) Training loop
for epoch in range(3):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            labels=batch['labels'].to(device)
        )
        outputs.loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            out = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device)
            )
            preds = torch.argmax(out.logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(batch['labels'].cpu().numpy())

    print(f"\nEpoch {epoch+1} Validation Report:")
    print(classification_report(
        all_labels,
        all_preds,
        target_names=[
            "Conflict-related",
            "Protests and Public Reactions",
            "Political Statements",
            "Irrelevant"
        ]
    ))

# Final Macro F1
final_f1 = f1_score(all_labels, all_preds, average='macro')
print("Final Macro F1:", final_f1)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1 Validation Report:
                               precision    recall  f1-score   support

             Conflict-related       0.67      0.60      0.63        10
Protests and Public Reactions       0.50      0.14      0.22         7
         Political Statements       0.00      0.00      0.00         7
                   Irrelevant       0.48      0.88      0.62        16

                     accuracy                           0.53        40
                    macro avg       0.41      0.40      0.37        40
                 weighted avg       0.45      0.53      0.45        40



/mnt/raid1/MSCs/AY2425/skaul/miniconda3/envs/scraper/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/raid1/MSCs/AY2425/skaul/miniconda3/envs/scraper/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/raid1/MSCs/AY2425/skaul/miniconda3/envs/scraper/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi


Epoch 2 Validation Report:
                               precision    recall  f1-score   support

             Conflict-related       0.50      1.00      0.67        10
Protests and Public Reactions       1.00      0.14      0.25         7
         Political Statements       0.67      0.29      0.40         7
                   Irrelevant       0.88      0.88      0.88        16

                     accuracy                           0.68        40
                    macro avg       0.76      0.58      0.55        40
                 weighted avg       0.77      0.68      0.63        40


Epoch 3 Validation Report:
                               precision    recall  f1-score   support

             Conflict-related       0.62      1.00      0.77        10
Protests and Public Reactions       1.00      0.43      0.60         7
         Political Statements       0.57      0.57      0.57         7
                   Irrelevant       0.93      0.81      0.87        16

                

## LLM

In [36]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import json
from sklearn.metrics import classification_report, f1_score
from collections import Counter

# 1) Load your articles
with open("articles_test.json") as f:
    articles = json.load(f)

# 2) Summarizer pipeline (CPU)
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=-1
)

# 3) Classifier (BloomZ-3B in 8-bit)
model_name = "bigscience/bloomz-3b"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)
classifier = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=16,
    do_sample=False
)

# 4) Label mapping
candidate_labels = [
    "Conflict-related",
    "Political Statements",
    "Protests and Public Reactions",
    "Irrelevant"
]
label_map = {
    "Conflict-related":              1,
    "Protests and Public Reactions": 2,
    "Political Statements":          3,
    "Irrelevant":                    4
}

# 5) Summarize + classify with guards
y_true, y_pred = [], []
for art in articles:
    full_text = art["title"] + "\n\n" + art.get("description","") + "\n\n" + art["text"]

    # Summarize (with fallback to first 200 chars if something goes wrong)
    try:
        summ_list = summarizer(full_text, max_length=200, min_length=100, do_sample=False)
        summary = summ_list[0]["summary_text"]
    except Exception:
        summary = full_text[:200]

    # Classify (with fallback to “Irrelevant” on any error)
    prompt = (
        "You are a news classification assistant.\n\n"
        "Classify this summary into ONE of: Conflict-related, Political Statements,\n"
        "Protests and Public Reactions, Irrelevant.\n\n"
        f"Summary:\n{summary}\n\nCategory:"
    )
    try:
        outputs = classifier(prompt)
        if outputs and "generated_text" in outputs[0]:
            out_text = outputs[0]["generated_text"]
            pred_label = out_text.split("Category:")[-1].strip().split("\n")[0]
        else:
            pred_label = "Irrelevant"
    except Exception:
        print(f"Error classifying article {art['title'][:30]}... - defaulting to Irrelevant")

    y_true.append(art["label"])
    y_pred.append(label_map.get(pred_label, 4))  # default to Irrelevant=4

# 6) Metrics
print("Prediction counts:", Counter(y_pred))
print("\n" + classification_report(
    y_true,
    y_pred,
    labels=[1,2,3,4],
    target_names=candidate_labels
))
print("Macro F1:", f1_score(y_true, y_pred, average="macro"))


Device set to use cpu
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


KeyboardInterrupt: 

## DeepSeek

In [ ]:
import requests, json, time
from collections import Counter
from sklearn.metrics import classification_report, f1_score

# Load the same data
with open("articles_test.json", encoding="utf-8") as f:
    articles = json.load(f)

# We’ll do just the first 20
articles20 = articles[:20]

# Your OpenRouter key here
API_KEY = "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"
URL     = "https://openrouter.ai/api/v1/chat/completions"

# Label mapping
label_map = {
    "Conflict-related":              1,
    "Protests and Public Reactions": 2,
    "Political Statements":          3,
    "Irrelevant":                    4
}
reverse_map = {v:k for k,v in label_map.items()}

def classify_one(article):
    # Build the prompt
    prompt = (
        "You are a news classification assistant.\n\n"
        "Classify into ONE category:\n"
        "1. Conflict-related  – News directly reporting on Israel–Palestine conflict events\n"
        "2. Political Statements  – Statements by world leaders on the conflict - Only inlcude if the article reports on political statement made directly relating to the israel hamas conflict\n"
        "3. Protests and Public Reactions  – Citizen protests or opinion pieces on the israel and hamas conflict or the larger war\n"
        "4. Irrelevant  – Not related to the conflict in any way\n\n"
        "Return ONLY the category name.\n\n"
        f"Title: {article['title']}\n\n"
        f"Content: {article['text']}"
    )
    payload = {
        "model":    "mistralai/mistral-small-3.2-24b-instruct:free",
        "messages": [{"role":"user","content":prompt}]
    }
    # Encode JSON as UTF-8 to avoid Latin-1 issues
    body = json.dumps(payload, ensure_ascii=False).encode("utf-8")
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json; charset=utf-8"
    }
    print(body)
    r = requests.post(URL, headers=headers, data=body)
    if r.status_code != 200:
        print(r.json())
        return 4
    reply = r.json()["choices"][0]["message"]["content"].strip().lower()
    for cat, idx in label_map.items():
        if cat.lower() in reply:
            return idx
    print(r.json())
    return 4

# Run classification
y_true, y_pred = [], []
for art in articles20:
    y_true.append(art["label"])
    y_pred.append(classify_one(art))
    time.sleep(1.2)  # keep under rate limits

# Print metrics
print("Counts:", Counter(y_pred))
print("\nClassification Report:")
print(classification_report(
    y_true, y_pred,
    labels=[1,2,3,4],
    target_names=list(label_map.keys())
))
print("Macro F1:", f1_score(y_true, y_pred, average="macro"))

# Print mismatches
print("\nMISMATCHED ARTICLES:")
for art, t, p in zip(articles20, y_true, y_pred):
    if t != p:
        print(f"- Title: {art['title']}")
        print(f"  URL:   {art.get('url','N/A')}")
        print(f"  True:  {reverse_map[t]}")
        print(f"  Pred:  {reverse_map[p]}\n")


b'{"model": "deepseek/deepseek-r1-0528:free", "messages": [{"role": "user", "content": "You are a news classification assistant.\\n\\nClassify into ONE category:\\n1. Conflict-related  \xe2\x80\x93 News directly reporting on Israel\xe2\x80\x93Palestine conflict events\\n2. Political Statements  \xe2\x80\x93 Statements by world leaders on the conflict - Only inlcude if the article reports on political statement made directly relating to the israel hamas conflict\\n3. Protests and Public Reactions  \xe2\x80\x93 Citizen protests or opinion pieces on the israel and hamas conflict or the larger war\\n4. Irrelevant  \xe2\x80\x93 Not related to the conflict in any way\\n\\nReturn ONLY the category name.\\n\\nTitle: Ramadan in Gaza: Palestinians fast amid war, devastation and looming famine\\n\\nContent: On the eve of  Ramadan , Hussein Owda and seven members of his family sat down for suhour \xe2\x80\x94 the meal Muslims eat before fasting from sunrise to sunset. On the table in front of them

KeyboardInterrupt: 

In [20]:
# Print mismatches
print("\nMISMATCHED ARTICLES:")
for art, t, p in zip(articles20, y_true, y_pred):
    if t != p:
        print(f"- Title: {art['title']}")
        print(f"  URL:   {art.get('url','N/A')}")
        print(f"  True:  {reverse_map[t]}")
        print(f"  Pred:  {reverse_map[p]}\n")



MISMATCHED ARTICLES:
- Title: Ramadan in Gaza: Palestinians fast amid war, devastation and looming famine
  URL:   https://www.nbcnews.com/news/world/ramadan-gaza-palestinians-israel-war-famine-rcna142785
  True:  Conflict-related
  Pred:  Irrelevant

- Title: UCLA anti-Israel protesters took hand-to-hand combat classes ahead of bloody clash
  URL:   https://nypost.com/2024/05/01/us-news/ucla-anti-israel-protesters-took-hand-to-hand-classes-ahead-of-bloody-clash/
  True:  Protests and Public Reactions
  Pred:  Irrelevant

- Title: Newspaper headlines: Conflicting Gaza death accounts and police vetting failures
  URL:   https://www.bbc.com/news/blogs-the-papers-68443012
  True:  Protests and Public Reactions
  Pred:  Irrelevant

- Title: Republicans blast Biden over plans to take in Gaza refugees: ‘Absolute disgrace’
  URL:   https://nypost.com/2024/05/01/us-news/republicans-blast-biden-over-plans-to-take-in-gaza-refugees-absolute-disgrace/
  True:  Political Statements
  Pred:  Irrele

In [35]:
import json, requests

API_KEY = "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"
URL     = "https://openrouter.ai/api/v1/chat/completions"

# Build your payload
prompt = (
    "You are a news classification assistant.\n\n"
    "Classify into ONE category:\n"
    "1. Conflict-related  - News directly reporting on Israel–Palestine events\n"
    "2. Political Statements  - Statements by world leaders on the conflict\n"
    "3. Protests and Public Reactions  - Citizen protests or opinion pieces\n"
    "4. Irrelevant  - Not related\n\n"
    "Return ONLY the category name.\n\n"
    "Title: Newspaper headlines: Conflicting Gaza death accounts and police vetting failures"
    "Content: The dramatic political comeback of George Galloway presents Sir Keir Starmer with another headache -  according to the Sun ,  external  in its online coverage.   The paper says the left-wing firebrand and former Big Brother contestant focused on Gaza as he gallivanted through mosques across Rochdale, to seize the support of its Muslim population.   The Times says ,  external  the campaign was one of the most divisive in recent years.   The Guardian uses ,  external  the same word to describe Mr Galloway.  It says he was so confident of victory that shortly after the polls closed, he\'d briefed reporters that he\'d won comfortably, before announcing plans for a mass rally.  The Daily Telegraph covers the spat ,  external  between Rochdale\'s new MP and the leader of the Reform Party, Richard Tice - who claims that his candidate and campaigners suffered abuse and death threats.  Mr Galloway rejects the suggestion his supporters engaged in bad behaviour - saying that Mr Tice had \\rather lost his balance\\.   The front page picture of  the Guardian captures  ,  external the aftermath of the chaos in Gaza.  It shows three Palestinian men holding a body, which is wrapped in a white sheet.  The paper says there are \\starkly different accounts\\ of how the deaths occurred.   The Times believes ,  external  the tragedy will almost certainly end hopes of a six week ceasefire.  In an analysis piece, it says Israel is dependent on the Biden administration for arms and diplomatic support - and the moment where America pushes the brakes seems near. There\'s widespread condemnation of the errors made by the Metropolitan Police, which led to the appointment of Wayne Couzens to the Diplomatic Protection Squad -- before he kidnapped and murdered the marketing executive, Sarah Everard.  Sarah Vine  in the Daily Mail says ,  external  it would have been hard-wired into Ms Everard\'s brain that a policeman was someone she could trust and that it sets her blood boiling, that she met a monster.   The Sun is equally condemnatory ,  external  describing officers responsible for vetting recruits a"
)

payload = {
    "model":    "mistralai/mistral-small-3.2-24b-instruct:free",     # or try "deepseek-r1" if that errors
    "messages": [{"role":"user","content":prompt}]
}

# Prepare request
body = json.dumps(payload, ensure_ascii=False).encode("utf-8")
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type":  "application/json; charset=utf-8"
}

# Send & debug
resp = requests.post(URL, headers=headers, data=body)
print("HTTP Status:", resp.status_code)
print("Response Body:")
print(resp.text)


HTTP Status: 200
Response Body:

         

         

         
{"id":"gen-1750634484-YYETfxaDDtDLp6JkFF2s","provider":"Chutes","model":"mistralai/mistral-small-3.2-24b-instruct:free","object":"chat.completion","created":1750634484,"choices":[{"logprobs":null,"finish_reason":"stop","native_finish_reason":"stop","index":0,"message":{"role":"assistant","content":"Conflict-related","refusal":null,"reasoning":null}}],"usage":{"prompt_tokens":537,"completion_tokens":3,"total_tokens":540}}


## Cerebras

In [ ]:

import os
import json
import time
from collections import Counter
from sklearn.metrics import classification_report, f1_score
from cerebras.cloud.sdk import Cerebras

# 1) Ensure your API key is set:
#    export CEREBRAS_API_KEY="your_real_key_here"
client = Cerebras(api_key="csk-3vy3dm6wyff5xpc8mjfy32498ew8cc5h4fw9cjxxyh3j5r95")

# 2) Load the first 20 articles
with open("articles_test.json", encoding="utf-8") as f:
    articles = json.load(f)
articles20 = articles[:20]

# 3) Define your label mappings
label_map = {
    "Conflict-related":              1,
    "Protests and Public Reactions": 2,
    "Political Statements":          3,
    "Irrelevant":                    4
}
reverse_map = {v: k for k, v in label_map.items()}

# 4) Classification function via Cerebras
def classify_one(article):
    system_msg = {
        "role": "system",
        "content": (
            "You are a precise news classification assistant. "
            "Respond with exactly one of: Conflict-related, Political Statements, "
            "Protests and Public Reactions, Irrelevant."
        )
    }
    user_msg = {
        "role": "user",
        "content": (
            "Classify into ONE category:\n"
            "1. Conflict-related – News directly reporting on Israel–Palestine conflict events\n"
            "2. Political Statements – Statements by world leaders on the conflict\n"
            "3. Protests and Public Reactions – Citizen protests or opinion pieces on the conflict\n"
            "4. Irrelevant – Not related to the conflict\n\n"
            f"Title: {article['title']}\n\n"
        )
    }

# f"Content: {article['text']}"
    resp = client.chat.completions.create(
        model="llama-4-scout-17b-16e-instruct",
        messages=[system_msg, user_msg],
        temperature=0.0
    )
    reply = resp.choices[0].message.content.strip().lower()
    for cat, idx in label_map.items():
        if cat.lower() in reply:
            return idx
    return label_map["Irrelevant"]  # fallback

# 5) Run classification and collect results
y_true, y_pred = [], []
for art in articles20:
    y_true.append(art["label"])
    y_pred.append(classify_one(art))
    time.sleep(1.2)  # rate-limit

# 6) Print evaluation
print("Prediction counts:", Counter(y_pred))
print("\nClassification Report:")
print(classification_report(
    y_true, y_pred,
    labels=[1,2,3,4],
    target_names=list(label_map.keys())
))
print("Macro F1:", f1_score(y_true, y_pred, average="macro"))

# 7) List mismatches
print("\nMISMATCHED ARTICLES:")
for art, t, p in zip(articles20, y_true, y_pred):
    if t != p:
        print(f"- Title: {art['title']}")
        print(f"  URL:   {art.get('url','N/A')}")
        print(f"  True:  {reverse_map[t]}")
        print(f"  Pred:  {reverse_map[p]}\n")


Prediction counts: Counter({1: 10, 4: 6, 2: 3, 3: 1})

Classification Report:
                               precision    recall  f1-score   support

             Conflict-related       0.30      1.00      0.46         3
Protests and Public Reactions       0.33      0.50      0.40         2
         Political Statements       1.00      0.33      0.50         3
                   Irrelevant       1.00      0.50      0.67        12

                     accuracy                           0.55        20
                    macro avg       0.66      0.58      0.51        20
                 weighted avg       0.83      0.55      0.58        20

Macro F1: 0.507051282051282

MISMATCHED ARTICLES:
- Title: Lakewood shooter ‘donated money to megachurch’ before attack that left son fighting for life
  URL:   https://www.independent.co.uk/news/world/americas/crime/lakewood-shooter-donated-megachurch-letter-b2495530.html
  True:  Irrelevant
  Pred:  Conflict-related

- Title: The New Hampshire-Can